In [ ]:
dataFolder = 'D:/Downloads/Results/Stage 3/' #replace with relevant data folder

personFileName = 'B1_Admission.csv' #replace with relevant file name for the Admission CSV

In [ ]:
from os import listdir

csvList = [dataFolder+item for item in listdir(dataFolder) if item.endswith('csv')]

In [ ]:
##Random Days, non season compliant
from random import randint
import csv

dateAnonDict = dict()

with open(dataFolder+personFileName, encoding="utf-8", newline='') as personCSV:
    reader = csv.reader(personCSV)
    for row in reader:
        try:
            personID = row[1]
            if personID not in dateAnonDict:
                dateAnonDict[personID] = randint(260, 521)*7 #+5 to 10 years, same weekday
        except IndexError:
            continue

In [ ]:
import csv
from datetime import datetime, timedelta
from re import findall

first = True
pattern = '\d+-\d+-\d+'
valueerrornum = 0

for file in csvList:
    with open(file, encoding="utf-8") as currFile:
        print(file)
        reader = csv.reader(currFile)
        with open(file.replace('.csv','')+'_Anon.csv', 'w+', encoding="utf-8") as newFile:
            for row in reader:
                new_row = []
                if first:
                    first = False
                    for column in row:
                        new_row.append('"%s"' % (column))
                    newFile.write(','.join(new_row)+'\n')
                    continue
                personID = row[1]
                
                for column in row:
                    length_column = len(column)
                    if length_column <= 23:
                        oldDates = findall(pattern, column)
                        newDates = [(datetime.strptime(item, '%Y-%m-%d') + 
                                     timedelta(dateAnonDict[personID])).strftime("%Y-%m-%d") for item in oldDates]
                    if newDates:
                        for i in range(len(newDates)):
                            if datetime.strptime(oldDates[i], '%Y-%m-%d').weekday() != datetime.strptime(newDates[i], '%Y-%m-%d').weekday():
                                diff = timedelta(datetime.strptime(oldDates[i], '%Y-%m-%d').weekday()-datetime.strptime(newDates[i], '%Y-%m-%d').weekday())
                                newDates[i] = (datetime.strptime(newDates[i], '%Y-%m-%d') + diff).strftime("%Y-%m-%d")
                            column = column.replace(oldDates[i], newDates[i])
                            
                    column = column.replace('"', "''")
                    new_row.append('"%s"' % (column))
                newFile.write(','.join(new_row)+'\n')